# Quantum Langevin Equations for Frequency Modulated OEM Systems

## Imports and Initialization

In [1]:
# dependencies
from IPython.display import display, Math
from sympy import conjugate, exp, Function, I, latex, sqrt, symbols
from sympy.physics.quantum import Commutator, Dagger, hbar

In [2]:
# time 
t = symbols('t', real=True, positive=True)
# laser
A_l, kappa, omega_l = symbols('A_{l}, \\kappa, \\omega_{l}', real=True, positive=True)
# cavity
gamma_0, l, omega_c = symbols('\\gamma_{0}, l, \\omega_{c}', real=True, positive=True)
Delta_0 = symbols('\\Delta^{(0)}', real=True)
Delta_t = Function('\\Delta', commutative=True, real=True)(t)
# mechanical
gamma_1, m, omega_0 = symbols('\\gamma_{1}, m, \\omega_{0}', real=True, positive=True)
# LC circuit
C_b, C_p, d, L, omega_1, r = symbols('C_{b}, C_{p}, d, L, \\omega_{1}, r', real=True, positive=True)
A_v, V_DC = symbols('A_{v}, V_{DC}', real=True)
# interaction
G, g_0, g_1 = symbols('G, g_{0}, g_{1}', real=True, positive=True)
G_0_t = Function('G_{0}', commutative=True, complex=True)(t)
G_0R_t = Function('G_{0R}', commutative=True, real=True)(t)
G_0I_t = Function('G_{0I}', commutative=True, real=True)(t)
G_10_t = Function('G_{10}', commutative=True, real=True)(t)
G_11_t = Function('G_{11}', commutative=True, real=True)(t)

# modes
a_t = Function('\\hat{a}', commutative=False)(t)
b_0_t = Function('\\hat{b}_{0}', commutative=False)(t)
b_1_t = Function('\\hat{b}_{1}', commutative=False)(t)

# quadratures
q_0_t = Function('\\hat{q}_{0}', commutative=False)(t)
p_0_t = Function('\\hat{p}_{0}', commutative=False)(t)
q_1_t = Function('\\hat{q}_{1}', commutative=False)(t)
p_1_t = Function('\\hat{p}_{1}', commutative=False)(t)

# dimensionless quadratures
Q_0_t = Function('\\hat{Q}_{0}', commutative=False)(t)
P_0_t = Function('\\hat{P}_{0}', commutative=False)(t)
Q_1_t = Function('\\hat{Q}_{1}', commutative=False)(t)
P_1_t = Function('\\hat{P}_{1}', commutative=False)(t)

# input noises
a_in_t = Function('\\hat{a}^{(in)}', commutative=False)(t)
b_0_in_t = Function('\\hat{b}_{0}^{(in)}', commutative=False)(t)
b_1_in_t = Function('\\hat{b}_{1}^{(in)}', commutative=False)(t)

# classical amplitudes
alpha_t = Function('\\alpha', commutative=True, complex=True)(t)
beta_0_t = Function('\\beta_{0}', commutative=True, complex=True)(t)
beta_1_t = Function('\\beta_{1}', commutative=True, complex=True)(t)

# quantum fluctuations
delta_a_t = Function('\\delta \\hat{a}', commutative=False)(t)
delta_b_0_t = Function('\\delta \\hat{b}_{0}', commutative=False)(t)
delta_b_1_t = Function('\\delta \\hat{b}_{1}', commutative=False)(t)

# fluctuations quadratures
delta_X_t = Function('\\delta \\hat{X}', commutativ=False)(t)
delta_Y_t = Function('\\delta \\hat{Y}', commutative=False)(t)
delta_Q_0_t = Function('\\delta \\hat{Q}_{0}', commutative=False)(t)
delta_P_0_t = Function('\\delta \\hat{P}_{0}', commutative=False)(t)
delta_Q_1_t = Function('\\delta \\hat{Q}_{1}', commutative=False)(t)
delta_P_1_t = Function('\\delta \\hat{P}_{1}', commutative=False)(t)

# quadrature noises
X_in_t = Function('\\hat{X}^{(in)}', commutative=False)(t)
Y_in_t = Function('\\hat{Y}^{(in)}', commutative=False)(t)
Q_0_in_t = Function('\\hat{Q}_{0}^{(in)}', commutative=False)(t)
P_0_in_t = Function('\\hat{P}_{0}^{(in)}', commutative=False)(t)
Q_1_in_t = Function('\\hat{Q}_{1}^{(in)}', commutative=False)(t)
P_1_in_t = Function('\\hat{P}_{1}^{(in)}', commutative=False)(t)

## Capacitance

In [3]:
# capacitance
expr_C_num = C_p * (1 - q_0_t / d) + C_b
expr_C_den = 1 - q_0_t / d
display(Math('\\frac{1}{C} = ' + latex(expr_C_den / expr_C_num)))

# substitute participation ratio
expr_C_num = (expr_C_num * r / C_b).subs(C_p, C_b * (1 - r) / r).expand() * C_b / r
expr_C_inv = expr_C_den / expr_C_num
display(Math('\\Rightarrow \\frac{1}{C} = ' + latex(expr_C_inv)))

# binomial approximation
expr_C_inv_approx = expr_C_inv.subs((1 + r * q_0_t / d - q_0_t / d)**-1, 1 - r * q_0_t / d + q_0_t / d).expand()
display(Math('\\Rightarrow \\frac{1}{C} \\approx ' + latex(expr_C_inv_approx)))

# neglect quadratic terms
expr_C_inv_approx = expr_C_inv_approx.subs(q_0_t**2 / d**2, 0).expand()
display(Math('\\Rightarrow \\frac{1}{C} \\approx ' + latex(expr_C_inv_approx)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Hamiltonian in the Interaction Picture

In [4]:
# laser Hamiltonian
expr_H_l = I * hbar * A_l * (Dagger(a_t) * exp(- I * omega_l * t) - a_t * exp(I * omega_l * t))
# cavity Hamiltonian
expr_H_c = hbar * omega_c * Dagger(a_t) * a_t
# mechanical Hamiltonian
expr_H_0 = m * omega_0**2 * q_0_t**2 / 2 + p_0_t**2 / 2 / m
# LC circuit Hamiltonian
expr_H_1 = r * q_1_t**2 / 2 / C_b + p_1_t**2 / 2 / L - q_1_t * V_DC
# interaction Hamiltonians
expr_H_om = - hbar * omega_c / l * q_0_t * Dagger(a_t) * a_t
expr_H_em = - r**2 / 2 / C_b / d * q_0_t * q_1_t**2
# total Hamiltonian
expr_H_tot = expr_H_c + expr_H_0 + expr_H_om + expr_H_1 + expr_H_em + expr_H_l
display(Math('H_{tot} = ' + latex(expr_H_tot)))

# hamiltonian in the interaction picture
list_subs = [
    (a_t, a_t * exp(- I * omega_l * t)), 
    (omega_c / l, G), 
    (omega_c, omega_l + Delta_0)
]
expr_H_int = expr_H_tot.subs(list_subs).expand() - hbar * omega_l * Dagger(a_t) * a_t
display(Math('\\Rightarrow H_{int} = ' + latex(expr_H_int)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Hamiltonian in terms of Dimensionless Quadratures

In [5]:
# zero point fluctuations
expr_q_0_zpf = sqrt(hbar / m / omega_0)
expr_p_0_zpf = sqrt(hbar * m * omega_0)
expr_q_1_zpf = sqrt(hbar / L / omega_1)
expr_p_1_zpf = sqrt(hbar * L * omega_1)
list_subs = [
    (q_0_t, expr_q_0_zpf * Q_0_t), 
    (p_0_t, expr_p_0_zpf * P_0_t), 
    (q_1_t, expr_q_1_zpf * Q_1_t), 
    (p_1_t, expr_p_1_zpf * P_1_t)
]
expr_H_dim = expr_H_int.subs(list_subs).expand()
display(Math('q_{0_{zpf}} = ' + latex(expr_q_0_zpf) + ', \\quad p_{0_{zpf}} = ' + latex(expr_p_0_zpf)))
display(Math('q_{1_{zpf}} = ' + latex(expr_q_1_zpf) + ', \\quad p_{1_{zpf}} = ' + latex(expr_p_1_zpf)))
display(Math('\\Rightarrow H_{dim} = ' + latex(expr_H_dim)))

# coupling coefficients
expr_g_0 = G * expr_q_0_zpf / sqrt(2)
expr_g_1 = expr_q_0_zpf * expr_q_1_zpf**2 * r**2 / 4 / sqrt(2) / hbar / C_b / d
expr_A_v = expr_q_1_zpf * V_DC / sqrt(2) / hbar
expr_L_inv = C_b / r * omega_1**2
list_subs = [
    (sqrt(2) * expr_g_0, sqrt(2) * g_0), 
    (2 * sqrt(2) * expr_g_1, 2 * sqrt(2) * g_1), 
    (sqrt(2) * hbar * expr_A_v, sqrt(2) * hbar * A_v), 
    (1 / L, expr_L_inv)
]
expr_H_dim = expr_H_dim.subs(list_subs).expand()
display(Math('g_{0} = ' + latex(expr_g_0) + ',\\quad g_{1} = ' + latex(expr_g_1) + ', \\quad A_{v} = ' + latex(expr_A_v)))
display(Math('\\Rightarrow H_{dim} \\approx ' + latex(expr_H_dim.collect([I * hbar * A_l, hbar * omega_0, hbar * omega_1]))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Hamiltonian in terms of the Modes

In [6]:
# energies and quadratures
list_subs = [
    (Q_0_t**2 / 2 + P_0_t**2 / 2, Dagger(b_0_t) * b_0_t),
    (Q_1_t**2 / 2 + P_1_t**2 / 2, Dagger(b_1_t) * b_1_t),
    (Q_0_t, (Dagger(b_0_t) + b_0_t) / sqrt(2)), 
    (P_0_t, I * (Dagger(b_0_t) - b_0_t) / sqrt(2)), 
    (Q_1_t, (Dagger(b_1_t) + b_1_t) / sqrt(2)), 
    (P_1_t, I * (Dagger(b_1_t) - b_1_t) / sqrt(2))
]
expr_H = expr_H_dim.collect([hbar * omega_0, hbar * omega_1]).subs(list_subs)
display(Math('H = ' + latex(expr_H.collect([I * hbar * A_l, - hbar * A_v]))))

<IPython.core.display.Math object>

## Heisenberg Equations of Motion

In [7]:
# mode rates
expr_da_dt_t = (- I / hbar * Commutator(a_t, expr_H)).doit().expand()
expr_db_0_dt_t = (- I / hbar * Commutator(b_0_t, expr_H)).doit().expand()
expr_db_1_dt_t = (- I / hbar * Commutator(b_1_t, expr_H)).doit().expand()

# commutation relations
list_subs = [
    (a_t * b_0_t, b_0_t * a_t), 
    (a_t * Dagger(b_0_t), Dagger(b_0_t) * a_t), 
    (a_t * b_1_t, b_1_t * a_t), 
    (a_t * Dagger(b_1_t), Dagger(b_1_t) * a_t),
    (Dagger(a_t) * b_0_t, b_0_t * Dagger(a_t)), 
    (Dagger(a_t) * Dagger(b_0_t), Dagger(b_0_t) * Dagger(a_t)), 
    (Dagger(a_t) * b_1_t, b_1_t * Dagger(a_t)), 
    (Dagger(a_t) * Dagger(b_1_t), Dagger(b_1_t) * Dagger(a_t)),
    (b_0_t * b_1_t, b_1_t * b_0_t), 
    (b_0_t * Dagger(b_1_t), Dagger(b_1_t) * b_0_t),
    (Dagger(b_0_t) * b_1_t, b_1_t * Dagger(b_0_t)), 
    (Dagger(b_0_t) * Dagger(b_1_t), Dagger(b_1_t) * Dagger(b_0_t))
]
# cavity mode
list_subs += [(a_t * Dagger(a_t), 1 + Dagger(a_t) * a_t)]
expr_da_dt_t = expr_da_dt_t.subs(list_subs).expand()
expr_da_dt_t = expr_da_dt_t.subs(list_subs).expand()
display(Math('\\frac{d \\dot{\\hat{a}}}{dt}(t) = ' + latex(expr_da_dt_t.collect(I * g_0))))
# mechanical mode
list_subs += [(b_0_t * Dagger(b_0_t), 1 + Dagger(b_0_t) * b_0_t)]
expr_db_0_dt_t = expr_db_0_dt_t.subs(list_subs).expand()
expr_db_0_dt_t = expr_db_0_dt_t.subs(list_subs).expand()
expr_db_0_dt_t = expr_db_0_dt_t.subs(list_subs).expand()
display(Math('\\frac{d \\dot{\\hat{b}}_0}{dt}(t) = ' + latex(expr_db_0_dt_t.collect(I * g_1))))
# LC circuit mode
list_subs += [(b_1_t * Dagger(b_1_t), 1 + Dagger(b_1_t) * b_1_t)]
expr_db_1_dt_t = expr_db_1_dt_t.subs(list_subs).expand()
expr_db_1_dt_t = expr_db_1_dt_t.subs(list_subs).expand()
expr_db_1_dt_t = expr_db_1_dt_t.subs(list_subs).expand()
display(Math('\\frac{d \\dot{\\hat{b}}_1}{dt}(t) = ' + latex(expr_db_1_dt_t.collect(2 * I * g_1))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Quantum Langevin Equations

In [8]:
# cavity decay and vaccum fluctuations
expr_da_dt_t_io = expr_da_dt_t - kappa * a_t + sqrt(2 * kappa) * a_in_t
display(Math('\\frac{d \\dot{\\hat{a}}}{dt}(t) = ' + latex(expr_da_dt_t_io.collect(I * g_0))))
# mechanical damping and thermal fluctuations
expr_db_0_dt_t_io = expr_db_0_dt_t - gamma_0 * b_0_t + sqrt(2 * gamma_0) * b_0_in_t
display(Math('\\frac{d \\dot{\\hat{b}}_0}{dt}(t) = ' + latex(expr_db_0_dt_t_io.collect(I * g_1))))
# electrical damping and thermal fluctuations
expr_db_1_dt_t_io = expr_db_1_dt_t - gamma_1 * b_1_t + sqrt(2 * gamma_1) * b_1_in_t
display(Math('\\frac{d \\dot{\\hat{b}}_1}{dt}(t) = ' + latex(expr_db_1_dt_t_io.collect(2 * I * g_1))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Classical Amplitudes

In [9]:
# effective detuning
list_subs = [
    (a_t, alpha_t), (a_in_t, 0),
    (b_0_t, beta_0_t), (b_0_in_t, 0),
    (b_1_t, beta_1_t), (b_1_in_t, 0),
    (Delta_0, Delta_t + g_0 * (beta_0_t + conjugate(beta_0_t)))
]
display(Math(latex(Delta_t) + ' = ' + latex(Delta_0 - g_0 * (beta_0_t + conjugate(beta_0_t)))))
# cavity mode
expr_dalpha_dt_t = expr_da_dt_t_io.subs(list_subs).expand()
display(Math('\\frac{d \\dot{\\alpha}}{dt}(t) = ' + latex(expr_dalpha_dt_t.collect(- alpha_t))))
# mechanical mode
expr_dbeta_0_dt_t = expr_db_0_dt_t_io.subs(list_subs).expand()
display(Math('\\frac{d \\dot{\\beta}_{0}}{dt}(t) = ' + latex(expr_dbeta_0_dt_t.collect([- beta_0_t, I * g_1]))))
# LC circuit mode
expr_dbeta_1_dt_t = expr_db_1_dt_t_io.subs(list_subs)
display(Math('\\frac{d \\dot{\\beta}_{1}}{dt}(t) = ' + latex(expr_dbeta_1_dt_t.collect([- beta_1_t, 2 * I * g_1]))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Quantum Fluctuations

In [10]:
# mode description
list_subs_0 = [
    (a_t, alpha_t + delta_a_t),
    (b_0_t, beta_0_t + delta_b_0_t),
    (b_1_t, beta_1_t + delta_b_1_t),
    (Delta_0, Delta_t + g_0 * (beta_0_t + conjugate(beta_0_t)))
]
display(Math(latex(Delta_t) + ' = ' + latex(Delta_0 - g_0 * (beta_0_t + conjugate(beta_0_t)))))
# linearization
list_subs_1 = [
    (delta_a_t * delta_a_t, 0),
    (delta_a_t * Dagger(delta_a_t), 0),
    (Dagger(delta_a_t) * delta_a_t, 0),
    (Dagger(delta_a_t) * Dagger(delta_a_t), 0),
    (delta_b_0_t * delta_a_t, 0),
    (delta_b_0_t * Dagger(delta_a_t), 0),
    (delta_b_0_t * delta_b_0_t, 0),
    (delta_b_0_t * Dagger(delta_b_0_t), 0),
    (Dagger(delta_b_0_t) * delta_a_t, 0),
    (Dagger(delta_b_0_t) * Dagger(delta_a_t), 0),
    (Dagger(delta_b_0_t) * delta_b_0_t, 0),
    (Dagger(delta_b_0_t) * Dagger(delta_b_0_t), 0),
    (delta_b_1_t * delta_a_t, 0),
    (delta_b_1_t * Dagger(delta_a_t), 0),
    (delta_b_1_t * delta_b_0_t, 0),
    (delta_b_1_t * Dagger(delta_b_0_t), 0),
    (delta_b_1_t * delta_b_1_t, 0),
    (delta_b_1_t * Dagger(delta_b_1_t), 0),
    (Dagger(delta_b_1_t) * delta_a_t, 0),
    (Dagger(delta_b_1_t) * Dagger(delta_a_t), 0),
    (Dagger(delta_b_1_t) * delta_b_0_t, 0),
    (Dagger(delta_b_1_t) * Dagger(delta_b_0_t), 0),
    (Dagger(delta_b_1_t) * delta_b_1_t, 0),
    (Dagger(delta_b_1_t) * Dagger(delta_b_1_t), 0)
]
# new variables
list_subs_2 = [
    (g_0 * alpha_t, G_0_t),
    (g_0 * conjugate(alpha_t), conjugate(G_0_t)),
    (g_1 * beta_0_t, G_10_t - g_1 * conjugate(beta_0_t)),
    (g_1 * beta_1_t, G_11_t - g_1 * conjugate(beta_1_t))
]
display(Math(latex(G_0_t) + ' = ' + latex(g_0 * alpha_t) + ', \\quad ' + latex(G_10_t) + ' = ' + latex(g_1 * (conjugate(beta_0_t) + beta_0_t)) + ', \\quad ' + latex(G_11_t) + ' = ' + latex(g_1 * (conjugate(beta_1_t) + beta_1_t))))
# common terms
list_coll = [
    I * G_0_t,
    I * g_0 * alpha_t, 
    I * g_0 * conjugate(alpha_t), 
    I * g_0, 
    2 * I * G_10_t, 
    2 * I * g_1 * beta_0_t, 
    2 * I * g_1 * conjugate(beta_0_t), 
    2 * I * G_11_t, 
    2 * I * g_1 * beta_1_t, 
    2 * I * g_1 * conjugate(beta_1_t), 
    2 * I * g_1,
    I * g_1
]
# cavity mode
expr_ddelta_a_dt_t = expr_da_dt_t_io.subs(list_subs_0).expand() - expr_dalpha_dt_t
display(Math('\\frac{d \\delta \\dot{a}}{dt}(t) = ' + latex(expr_ddelta_a_dt_t.collect(list_coll))))
expr_ddelta_a_dt_t_lin = expr_ddelta_a_dt_t.subs(list_subs_1).expand().subs(list_subs_2).expand()
display(Math('\\Rightarrow \\frac{d \\delta \\dot{a}}{dt}(t) \\approx ' + latex(expr_ddelta_a_dt_t_lin.collect(list_coll))))
# mechanical mode
expr_ddelta_b_0_dt_t = expr_db_0_dt_t_io.subs(list_subs_0).expand() - expr_dbeta_0_dt_t.expand()
display(Math('\\frac{d \\delta \\dot{b}_{0}}{dt}(t) = ' + latex(expr_ddelta_b_0_dt_t.collect(list_coll))))
expr_ddelta_b_0_dt_t_lin = expr_ddelta_b_0_dt_t.subs(list_subs_1).expand().subs(list_subs_2).expand()
display(Math('\\Rightarrow \\frac{d \\delta \\dot{b}_{0}}{dt}(t) \\approx ' + latex(expr_ddelta_b_0_dt_t_lin.collect(list_coll))))
# LC circuit mode
expr_ddelta_b_1_dt_t = expr_db_1_dt_t_io.subs(list_subs_0).expand() - expr_dbeta_1_dt_t.expand()
display(Math('\\frac{d \\delta \\dot{b}_{1}}{dt}(t) = ' + latex(expr_ddelta_b_1_dt_t.collect(list_coll))))
expr_ddelta_b_1_dt_t_lin = expr_ddelta_b_1_dt_t.subs(list_subs_1).expand().subs(list_subs_2).expand()
display(Math('\\Rightarrow \\frac{d \\delta \\dot{b}_{1}}{dt}(t) \\approx ' + latex(expr_ddelta_b_1_dt_t_lin.collect(list_coll))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Fluctuation Quadratures

In [11]:
# quadratures
expr_delta_a_t = (delta_X_t + I * delta_Y_t) / sqrt(2)
expr_delta_b_0_t = (delta_Q_0_t + I * delta_P_0_t) / sqrt(2)
expr_delta_b_1_t = (delta_Q_1_t + I * delta_P_1_t) / sqrt(2)
display(Math(latex(delta_a_t) + ' = ' + latex(expr_delta_a_t) + ', \\quad ' + latex(delta_b_0_t) + ' = ' + latex(expr_delta_b_0_t) + ', \\quad ' + latex(delta_b_1_t) + ' = ' + latex(expr_delta_b_1_t)))
# noises
expr_a_in_t = (X_in_t + I * Y_in_t) / sqrt(2)
expr_b_0_in_t = (Q_0_in_t + I * P_0_in_t) / sqrt(2)
expr_b_1_in_t = (Q_1_in_t + I * P_1_in_t) / sqrt(2)
display(Math(latex(a_in_t) + ' = ' + latex(expr_a_in_t) + ', \\quad ' + latex(b_0_in_t) + ' = ' + latex(expr_b_0_in_t) + ', \\quad ' + latex(b_1_in_t) + ' = ' + latex(expr_b_1_in_t)))
# substitutions
list_subs = [
    (G_0_t, G_0R_t + I * G_0I_t),
    (delta_a_t, expr_delta_a_t),
    (a_in_t, expr_a_in_t),
    (Dagger(delta_X_t), delta_X_t),
    (Dagger(delta_Y_t), delta_Y_t),
    (Dagger(X_in_t), X_in_t),
    (Dagger(Y_in_t), Y_in_t),
    (delta_b_0_t, expr_delta_b_0_t),
    (b_0_in_t, expr_b_0_in_t),
    (Dagger(delta_Q_0_t), delta_Q_0_t),
    (Dagger(delta_P_0_t), delta_P_0_t),
    (Dagger(Q_0_in_t), Q_0_in_t),
    (Dagger(P_0_in_t), P_0_in_t),
    (delta_b_1_t, expr_delta_b_1_t),
    (b_1_in_t, expr_b_1_in_t),
    (Dagger(delta_Q_1_t), delta_Q_1_t),
    (Dagger(delta_P_1_t), delta_P_1_t),
    (Dagger(Q_1_in_t), Q_1_in_t),
    (Dagger(P_1_in_t), P_1_in_t)
]
# cavity quadratures
expr_ddelta_X_dt_t = (Dagger(expr_ddelta_a_dt_t_lin) + expr_ddelta_a_dt_t_lin) / sqrt(2)
expr_ddelta_X_dt_t = expr_ddelta_X_dt_t.subs(list_subs).expand()
display(Math('\\frac{d \\delta \\dot{X}}{dt}(t) = ' + latex(expr_ddelta_X_dt_t)))
expr_ddelta_Y_dt_t = I * (Dagger(expr_ddelta_a_dt_t_lin) - expr_ddelta_a_dt_t_lin) / sqrt(2)
expr_ddelta_Y_dt_t = expr_ddelta_Y_dt_t.subs(list_subs).expand()
display(Math('\\frac{d \\delta \\dot{Y}}{dt}(t) = ' + latex(expr_ddelta_Y_dt_t)))
# mechanical quadratures
expr_ddelta_Q_0_dt_t = (Dagger(expr_ddelta_b_0_dt_t_lin) + expr_ddelta_b_0_dt_t_lin) / sqrt(2)
expr_ddelta_Q_0_dt_t = expr_ddelta_Q_0_dt_t.subs(list_subs).expand()
display(Math('\\frac{d \\delta \\dot{Q}_{0}}{dt}(t) = ' + latex(expr_ddelta_Q_0_dt_t)))
expr_ddelta_P_0_dt_t = I * (Dagger(expr_ddelta_b_0_dt_t_lin) - expr_ddelta_b_0_dt_t_lin) / sqrt(2)
expr_ddelta_P_0_dt_t = expr_ddelta_P_0_dt_t.subs(list_subs).expand()
display(Math('\\frac{d \\delta \\dot{P}_{0}}{dt}(t) = ' + latex(expr_ddelta_P_0_dt_t)))
# LC circuit quadratures
expr_ddelta_Q_1_dt_t = (Dagger(expr_ddelta_b_1_dt_t_lin) + expr_ddelta_b_1_dt_t_lin) / sqrt(2)
expr_ddelta_Q_1_dt_t = expr_ddelta_Q_1_dt_t.subs(list_subs).expand()
display(Math('\\frac{d \\delta \\dot{Q}_{1}}{dt}(t) = ' + latex(expr_ddelta_Q_1_dt_t)))
expr_ddelta_P_1_dt_t = I * (Dagger(expr_ddelta_b_1_dt_t_lin) - expr_ddelta_b_1_dt_t_lin) / sqrt(2)
expr_ddelta_P_1_dt_t = expr_ddelta_P_1_dt_t.subs(list_subs).expand()
display(Math('\\frac{d \\delta \\dot{P}_{1}}{dt}(t) = ' + latex(expr_ddelta_P_1_dt_t)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>